In [4]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()



True

In [2]:
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")



In [6]:
llm = ChatGroq(
    model_name="mistral-saba-24b",  # Updated to current Groq model name
    temperature=0.7,
    max_tokens=4096,
 
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e9c22107b00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e9c22124c50>, model_name='mistral-saba-24b', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=4096)

In [8]:
from langchain_core.messages import HumanMessage
result=llm.invoke([HumanMessage(content="Hi .My name is Aryaman, I am an AI engineer")])
print(result.content)

Hello Aryaman! Nice to meet you. How can I assist you today? If you're in the field of AI, we can discuss anything from machine learning algorithms to AI ethics, or even some of the latest trends in the industry. What's on your mind?


In [11]:
from langchain_core.messages import AIMessage

res=llm.invoke([
    HumanMessage(content="Hi .My name is Aryaman, I am an AI engineer"),
    AIMessage(content="Hello Aryaman! Nice to meet you. How can I assist you today? If you're in the field of AI, we can discuss anything from machine learning algorithms to AI ethics, or even some of the latest trends in the industry. What's on your mind?"),
    HumanMessage(content="Hey,what's my name and what do I do?")
    ])
print(res.content)

You've introduced yourself as Aryaman, and you mentioned that you are an AI engineer. Is there something specific you would like to talk about or learn more about in the field of AI?


In [20]:
## message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
#to know that different session we are buliding a function

store={}# define store as dict

def get_session(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(llm,get_session)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session at 0x7e9c21e66ac0>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [21]:
config={"configurable":{"session_id":"chat1"}}

In [23]:
resi=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Aryaman and I am an AI engineer")],
    config=config
)
print(resi.content)

Hello Aryaman! It's nice to meet you. As an AI engineer, you're likely involved in creating and refining AI systems. Here are a few things we could discuss or work on:

1. **Project Ideas**: I can help brainstorm project ideas that align with your interests and skills.
2. **Technical Guidance**: If you're working on a specific problem, I can provide technical guidance or suggest relevant resources.
3. **Algorithm Explanations**: I can help explain how certain algorithms work or suggest appropriate algorithms for specific tasks.
4. **Best Practices**: We can discuss best practices in AI development, such as model evaluation, bias mitigation, or ethical considerations.
5. **Learning Resources**: If you're looking to expand your knowledge, I can recommend books, online courses, or research papers.
6. **Code Review**: If you're comfortable sharing code snippets, I can provide feedback and suggestions for improvement.

What would you like to focus on?


In [24]:
resp=with_message_history.invoke(
    [HumanMessage(content="tell my name and what is my occupation?")],
    config=config
)
print(resp.content)

Your name is Aryaman, and your occupation is an AI engineer.


In [30]:
#change -->the session id
config_1={"configurable":{"session_id":"chat2"}}

In [31]:
resp=with_message_history.invoke(
    
        [HumanMessage(content="tell my name")],
        config=config_1
    
)
print(resp.content)

I'm an assistant that operates on the information I've been trained on up until 2023, so I don't have real-time knowledge or personal experiences. I don't know your name. If you'd like, you can tell me your name and I can use it during our conversation. How about you? What's your name?


In [32]:
resp=with_message_history.invoke(
    
        [HumanMessage(content="hey my name is Aryaman")],
        config=config_1
    
)
print(resp.content)

Hello Aryaman! Nice to meet you. How are you today?


### Prompt Template



In [36]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        (
        "system","you are helpful assisant .Answer all the question to the best of your ablity",
),
        MessagesPlaceholder(variable_name="messages"),
    ]
    
)
chain=prompt|llm
chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [39]:
res_1=chain.invoke({"messages":[HumanMessage(content="Hi my name is Aryaman")]})
print(res_1.content)

Hello Aryaman! Nice to meet you. How can I assist you today?


In [41]:
with_message_history=RunnableWithMessageHistory(chain,get_session)

config_2={"configurable":{"session_id":"chat3"}}


In [42]:
with_message_history.invoke(
    [HumanMessage(content="Hi my name is Aryaman")],
    config=config_2
    
)


AIMessage(content="Hello Aryaman! It's nice to meet you. I'm here to help with any questions or tasks you have. Here are a few things I can do:\n\n1. **Answer Questions**: I can provide information based on the data I've been trained on (up to 2021).\n2. **Explain Concepts**: I can help explain complex ideas in a simpler way.\n3. **Provide Suggestions**: Whether it's a book to read, a movie to watch, or a recipe to cook, I can provide recommendations.\n4. **Help with Language**: I can help with language translation in multiple languages, define words, or provide synonyms/antonyms.\n5. **Perform Simple Tasks**: I can do simple calculations, conversions, or other basic tasks.\n\nWhat would you like help with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 28, 'total_tokens': 199, 'completion_time': 0.518181818, 'prompt_time': 0.002543717, 'queue_time': 0.099414714, 'total_time': 0.520725535}, 'model_name': 'mistral-saba-24b', '

In [52]:
## add more complexity

prompt=ChatPromptTemplate.from_messages(
    [
(
    "system","You are a helpful assistant.Answer all the question to the best of your ablity in {language}"
),
MessagesPlaceholder(variable_name="messages")

    ]
)
chain=prompt|llm
chain


ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langch

In [53]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Aryaman")],"language":"Hindi"})
response.content


'नमस्ते, आर्यमन! आपसे मिलकर खुशी हुई। मैं आपकी कैसे मदद कर सकता हूँ?'

In [56]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session,
    input_messages_key="messages"
)


In [66]:
config_3={"configurable":{"session_id":"chat4"}}

response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I Am Aryaman Tiwari")],"language":"Hindi"},
    config=config_3
    
)
response.content

'नमस्ते आर्यमन तिवारी! मैं आपकी मदद करने के लिए यहाँ हूँ। आप मुझे किसी भी सवाल पूछ सकते हैं, और मैं अपनी पूरी क्षमता से उसका उत्तर देने की कोशिश करूंगा।\n\nचूंकि मुझे आपके पेशे या काम के बारे में जानकारी नहीं है, इसलिए मैं आपसे यह पूछना चाहता हूँ कि आप क्या करते हैं? अगर आप मुझे बताएं, तो मैं आपको बेहतर ढंग से सहायता कर सकता हूँ।\n\nअगर आपको कोई और सवाल है या मदद की जरूरत है, तो बेझिजक पूछें। मैं आपकी मदद करने के लिए यहाँ हूँ!'

In [67]:
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="What is my name and what work I do?")],"language":"Hindi"},
    config=config_3
    
)
response.content

'नमस्ते आर्यमन तिवारी!\n\nचूंकि आपने मुझसे कहा है कि आप आर्यमन तिवारी हैं, इसलिए मुझे पता है कि आपका नाम आर्यमन तिवारी है। आपके पेशे या काम के बारे में मुझे कोई जानकारी नहीं है। अगर आप मुझे अपने पेशे या काम के बारे में बताना चाहें, तो मैं आपकी मदद कर सकता हूँ।\n\nअगर आपके पास कोई और सवाल है या किसी भी विषय पर मदद चाहिए, तो कृपया मुझे बताएं। मैं आपकी मदद करने के लिए यहाँ हूँ!'

## Manage the Conversation History


In [81]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
    
    
)
messages=[
    SystemMessage(content="youre a good assitant"),
    HumanMessage(content="hi I m AT"),
    AIMessage(content="hi"),
    HumanMessage(content="I like vanilla Icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    AIMessage(content="Thanks"),
    SystemMessage(content="having fun"),
    AIMessage(content="yes!")
]
res=trimmer.invoke(messages)

res

[SystemMessage(content='youre a good assitant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi I m AT', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla Icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='having fun', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [82]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | llm
)
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do I like")],
    "language":"English"
    }
)
response.content

'Based on our conversation, you mentioned that you like vanilla ice cream.'

In [85]:
## lets wrap this in the message history
with_message_history=RunnableWithMessageHistory(
    chain,get_session,input_messages_key="messages"
    
)
config_5={"configurable":{"session_id":"chat5"}}



In [87]:
response=with_message_history.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do I like")],
    "language":"English"
    },
    config=config_5
)
response.content

'You said you like vanilla ice cream! Is there a specific flavor or brand that you enjoy the most?'